In [1]:
import collections
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import math
import torchvision
from torch import  nn
from tqdm import tqdm
from torch import nn
from time import time as  time
import pandas as pd

In [2]:

class My_dataloader(Dataset):
    def __init__(self,data_dir,transform,filename_list,label_list = None,mode = 'train'):
        self.data_dir = data_dir
        self.transform = transform
        self.data_filename = filename_list
        self.mode = mode
        if self.mode == 'train':
            self.label_list = torch.tensor(label_list)    #label是一个数字列表
    def __getitem__(self, item):
        file_path = os.path.join(self.data_dir, list(self.data_filename)[item])
        img = Image.open(file_path)
        img = self.transform(img)
        if self.mode == 'train':
            label = self.label_list[item]
            return img,label
        else:
            return img
    def __len__(self):
        return len(self.data_filename)

def read_train_labels(fname):
    """读取 `fname` 来给标签字典返回一个文件名。"""
    with open(fname, 'r') as f:
        # 跳过文件头行 (列名)
        lines = f.readlines()[1:]
    tokens = [l.rstrip().split(',') for l in lines]
    return dict(((name, label) for name, label in tokens))

def read_test(fname):
    with open(fname, 'r') as f:
        # 跳过文件头行 (列名)
        lines = f.readlines()[1:]
    tokens = [l.rstrip().split(',')[0] for l in lines]
    return tokens

def class_to_num(data_dict):
    classes = sorted(set(data_dict.values()))
    num = [l for l in range(len(classes))]
    return dict(zip(classes,num))

def num_to_class(data_dict):
    classes = sorted(set(data_dict.values()))
    num = [l for l in range(len(classes))]
    return dict(zip(num,classes))

def tain_val_list(data_dict,class_num,valid_ratio):
    n = collections.Counter(data_dict.values()).most_common()[-1][1]
    n_valid_per_label = max(1, math.floor(n * valid_ratio))
    train_dict = {}
    val_dict = {}
    count = {}
    for key,value in data_dict.items():
        if value not in count or count[value] < n_valid_per_label:
            val_dict[key] = class_num[value]
            count[value] = count.get(value, 0) + 1
        else:
            train_dict[key] = class_num[value]
    return  train_dict,val_dict

def test(data_dir,train_csv_filename,test_csv_filename):
    train_all = read_train_labels(os.path.join(data_dir,train_csv_filename))
    test_all = read_test(os.path.join(data_dir,test_csv_filename))
    class_num = class_to_num(train_all)
    num_class = num_to_class(train_all)
    train_dict,val_dict = tain_val_list(train_all,class_num,0.1)
    # print(list(val_dict.values()))
    train_list = train_dict.keys()
    train_label = list(train_dict.values())
    val_list = val_dict.keys()
    val_label = list(val_dict.values())
    # print(len(train_list),len(train_label),len(test_list),len(test_label))
    train_loader = My_dataloader(data_dir,torchvision.transforms.ToTensor(),train_list,train_label)
    val_loader = My_dataloader(data_dir, torchvision.transforms.ToTensor(), val_list, val_label)
    test_loader = My_dataloader(data_dir,torchvision.transforms.ToTensor(),test_all,mode='test')
    return train_loader,val_loader,test_loader,num_class


def transform_convert(img_tensor, transform):
    """
    param img_tensor: tensor
    param transforms: torchvision.transforms
    """
    if 'Normalize' in str(transform):
        normal_transform = list(filter(lambda x: isinstance(x, transforms.Normalize), transform.transforms))
        mean = torch.tensor(normal_transform[0].mean, dtype=img_tensor.dtype, device=img_tensor.device)
        std = torch.tensor(normal_transform[0].std, dtype=img_tensor.dtype, device=img_tensor.device)
        img_tensor.mul_(std[:, None, None]).add_(mean[:, None, None])

    img_tensor = img_tensor.transpose(0, 2).transpose(0, 1)  # C x H x W  ---> H x W x C

    if 'ToTensor' in str(transform) or img_tensor.max() < 1:
        img_tensor = img_tensor.detach().numpy() * 255

    if isinstance(img_tensor, torch.Tensor):
        img_tensor = img_tensor.numpy()

    if img_tensor.shape[2] == 3:
        img = Image.fromarray(img_tensor.astype('uint8')).convert('RGB')
    elif img_tensor.shape[2] == 1:
        img = Image.fromarray(img_tensor.astype('uint8')).squeeze()
    else:
        raise Exception("Invalid img shape, expected 1 or 3 in axis 2, but got {}!".format(img_tensor.shape[2]))

    return img



def train(net,train_iter,test_iter,epochs,lr,device):
    net.to(device)
    loss = nn.CrossEntropyLoss()
    optim = torch.optim.Adam(net.parameters(),lr = lr)
    for epoch in range(epochs):
        start = time()
        train_loss = []
        train_accs = []
        for batch in tqdm(train_iter):
            X,y = batch
            X,y = X.to(device),y.to(device)
            y_hat = net(X)
            l = loss(y_hat,y)
            optim.zero_grad()
            l.backward()
            optim.step()
            acc = (y_hat.argmax(dim=-1) == y).float().mean()
            train_loss.append(l.item())
            train_accs.append(acc)
        train_loss = sum(train_loss) / len(train_loss)
        train_accs = sum(train_accs) / len(train_accs)

        test_loss = []
        test_accs = []
        net.eval()
        for batch in tqdm(test_iter):
            X,y = batch
            X,y = X.to(device),y.to(device)
            with torch.no_grad():
                y_hat = net(X)
            l = loss(y_hat,y)
            acc = (y_hat.argmax(dim=-1) == y).float().mean()
            test_loss.append(l.item())
            test_accs.append(acc)
        test_loss = sum(test_loss) / len(test_loss)
        test_accs = sum(test_accs) / len(test_accs)
        print('  ')
        print(f"[  Time | {epoch + 1:03d}/{epochs:03d} ] time = {time() - start:.5f}")
        print(f"[ Train | {epoch + 1:03d}/{epochs:03d} ] loss = {train_loss:.5f}, acc = {train_accs:.5f}")
        print(f"[ Valid | {epoch + 1:03d}/{epochs:03d} ] loss = {test_loss:.5f}, acc = {test_accs:.5f}")


def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

def pred(net,test_iter,filename,num_class,device,saveFileName):
    net.eval()
    predictions = []
    preds = []
    for X in tqdm(test_iter):
        X = X.to(device)
        with torch.no_grad():
            y_hat = net(X)
        predictions.extend(y_hat.argmax(dim = -1).cpu().numpy().tolist())
    for i in predictions:
        preds.append(num_class[i])
    test_data = pd.read_csv(filename)
    test_data['label'] = pd.Series(preds)
    submission = pd.concat([test_data['image'], test_data['label']], axis=1)
    submission.to_csv(saveFileName, index=False)



In [3]:
net =torchvision.models.alexnet(pretrained=True)
net.classifier = nn.Sequential(
    nn.Linear(9216,1024),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(1024,176),
    nn.LogSoftmax(dim=1)
)
net

In [ ]:
device = get_device()

data_dir = '../input/classify-leaves'
train_csv_filename = 'train.csv'
test_csv_filename = 'test.csv'
saveFileName = 'my_first_sub.csv'
batch_size = 1024

train_loader,val_loader,test_loader,num_class= test(data_dir, train_csv_filename, test_csv_filename)
train_iter = torch.utils.data.DataLoader(train_loader, batch_size, shuffle=True,drop_last=True)
valid_iter = torch.utils.data.DataLoader(val_loader, batch_size, shuffle=False,drop_last=False)
test_iter = torch.utils.data.DataLoader(test_loader, batch_size, shuffle=False,drop_last=False)


epoch = 1
lr = 0.001

train(net,train_iter,valid_iter,epoch,lr,device)
pred(net,test_iter,os.path.join(data_dir,test_csv_filename),num_class,device,saveFileName)
